In [86]:
import os
import xlrd
import re
import shutil
import sys
import threading
import time

import pdfplumber

import pyzbar.pyzbar as pyzbar
import numpy as np
import cv2

In [114]:
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument, PDFNoOutlines
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams, LTTextBox, LTTextLine, LTFigure, LTImage, LTChar

def parse_pdf_with_PdfMiner3K(path):
    with open(path, 'rb') as fp:
        parser = PDFParser(fp)
        doc = PDFDocument(parser)
        parser.set_document(doc)
#         doc.set_parser(parser)
#         doc.initialize('')
        rsrcmgr = PDFResourceManager()
        laparams = LAParams()
        laparams.char_margin = 1.0
        laparams.word_margin = 1.0
        device = PDFPageAggregator(rsrcmgr, laparams=laparams)
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        extracted_text = ''
        for i,page in enumerate(PDFPage.create_pages(doc)):
            interpreter.process_page(page)
            layout = device.get_result()
            for lt_obj in layout:
                if isinstance(lt_obj, LTTextBox) or isinstance(lt_obj, LTTextLine):
                    extracted_text += lt_obj.get_text()
    return extracted_text
#     with open(output_path, "w", encoding="utf-8") as f:
#         f.write(extracted_text)

In [110]:
def parse_pdf(path):
    with pdfplumber.open(path) as pdf:
#         first_page = pdf.pages[0]
#         print(first_page.chars[0])

        for page in pdf.pages:
            # 获取当前页面的全部文本信息，包括表格中的文字
            # print(page.extract_text())                        
        
            for table in page.extract_tables():
#                 print(table)
                for row in table:
                    print(row)
                    pass
                # print('---------- 分割线 ----------')
        
        pdf.close()
        result = {'pages': pdf.pages}
        return result

In [190]:
pdf_file_name = '日用品20190322_98p99'
#     pdf_file_name = '日用品20190417_197p98'
#     pdf_file_name = '食品20190408_91p8'
path = 'data\\invoice\\{pdf_file_name}.pdf'.format(pdf_file_name=pdf_file_name)
output_path = 'data\\result\\{pdf_file_name}.txt'.format(pdf_file_name=pdf_file_name)
result = parse_pdf(path)

['购\n买\n方', '名        称:北京慧远云程科技有限公司\n91110105306602928Y\n纳税人识别号:\n地 址、电 话:\n开户行及账号:', None, None, None, None, '密\n码\n区', '59+8>9232--*+99+333*-<9/1/-\n60/4</2*/137<6/0110*0/2<2</\n/0+9</9*<+*6*1+1+19>+130>-9\n*09>12>5+/1+/0-><>/0998+95+', None, None, None]
['货物或应税劳务、服务名称     \n*纺织产品*天石（HighRock）户外\n露营情侣左右互拼成人棉睡袋 -10度 \n右开 天蓝色\n*纺织产品*天石（HighRock）户外\n露营情侣左右互拼成人棉睡袋 -10度 \n右开 天蓝色\n合       计', None, '规格型号 \n冷杉', '单位', '数 量\n1', '单 价\n171.55', None, None, '金  额  \n171.55\n-86.22\n¥85.33', '税率\n16%\n16%', '税    额\n27.45\n-13.79\n¥13.66']
['价税合计(大写)', None, '¥98.99\n玖拾捌圆玖角玖分 (小写)', None, None, None, None, None, None, None, None]
['销\n售\n方', '名        称:沈阳京东世纪贸易有限公司\n912101125893550575\n纳税人识别号:\n地 址、电 话:辽宁省沈阳市浑南区麦子屯590-4号102室 024-67859504\n开户行及账号:中国建设银行沈阳市中山支行 21001360008059003247', None, None, None, None, '备\n注', '订单号:90235270381', None, None, None]


In [115]:
result2 = parse_pdf_with_PdfMiner3K(path)

In [116]:
result2

' 辽宁增值税电子普通发票\n监票发一统\n制\n国\n全\n国\n家税务\n总\n局\n章\n宁辽\n省税务\n局\n发票代码:\n021001900111\n发票号码:\n23763823\n开票日期:\n2019  03  22\n年\n月\n日\n机器编号:\n661807100422\n北京慧远云程科技有限公司\n91110105306602928Y\n购\n买\n方\n名        称:\n纳税人识别号:\n地 址、电 话:\n开户行及账号:\n校 验 码:\n70446  05854  14159  09133  \n59+8>9232--*+99+333*-<9/1/-\n60/4</2*/137<6/0110*0/2<2</\n/0+9</9*<+*6*1+1+19>+130>-9\n*09>12>5+/1+/0-><>/0998+95+\n密\n码\n区\n货物或应税劳务、服务名称      规格型号   单位\n 数 量\n   单 价\n金  额\n  税率          税  额\n*纺织产品*天石（HighRock）户外\n露营情侣左右互拼成人棉睡袋 -10度 \n右开 天蓝色\n*纺织产品*天石（HighRock）户外\n露营情侣左右互拼成人棉睡袋 -10度 \n右开 天蓝色\n冷杉\n1\n171.55\n171.55\n16%\n-86.22\n16%\n合       计\n价税合计(大写)\n玖拾捌圆玖角玖分\n¥85.33\n(小写)\n¥98.99\n销\n售\n方\n名        称:\n纳税人识别号:\n地 址、电 话:\n开户行及账号:\n沈阳京东世纪贸易有限公司\n912101125893550575\n辽宁省沈阳市浑南区麦子屯590-4号102室 024-67859504\n中国建设银行沈阳市中山支行 21001360008059003247\n备\n注\n订单号:90235270381\n收款人:\n京东商城\n复核:\n开票人:\n京东商城\n销售方:(章)\n27.45\n-13.79\n¥13.66\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \

In [78]:
def decode(im) : 
  # Find barcodes and QR codes
  decodedObjects = pyzbar.decode(im)
 
  # Print results
  for obj in decodedObjects:
    print('Type : ', obj.type)
    print('Data : ', obj.data.decode(encoding='utf-8'),'\n')
     
  return decodedObjects
# Display barcode and QR code location  
def display(im, decodedObjects):
 
  # Loop over all decoded objects
  for decodedObject in decodedObjects: 
    points = decodedObject.polygon
 
    # If the points do not form a quad, find convex hull
    if len(points) > 4 : 
      hull = cv2.convexHull(np.array([point for point in points], dtype=np.float32))
      hull = list(map(tuple, np.squeeze(hull)))
    else : 
      hull = points;
     
    # Number of points in the convex hull
    n = len(hull)
 
    # Draw the convext hull
    for j in range(0,n):
      cv2.line(im, hull[j], hull[ (j+1) % n], (255,0,0), 3)
 
  # Display results 
  cv2.imshow("Results", im);
  cv2.waitKey(0);

In [79]:
import qrcode
img = qrcode.make('hello, 中文qrcode测试')
img.save('zbar-test.png')

In [80]:
# Read image
im = cv2.imread('zbar-test.png')

decodedObjects = decode(im)
display(im, decodedObjects)

Type :  QRCODE
Data :  hello, 中文qrcode测试 



In [81]:
import re
  
line = "IF_MIB::=Counter32: 12345\nIF_MIB::=Counter32: 1234556";
result = re.findall( r'(?<=\:\s)\d+$', line, re.M)

In [82]:
result

['12345', '1234556']

In [83]:
import fitz
from io import BytesIO
from PIL import Image

In [84]:
doc = fitz.open(path)
# png = doc[0].getPixmap(matrix=fitz.Matrix(2,2), alpha=False).getPNGdata()
# img = Image.open(BytesIO(png))
# img.save('png_from_pdf.png')
# img.show()
pm = doc[0].getPixmap(matrix=fitz.Matrix(2,2), alpha=False)
pm.writePNG('png_from_pdf.png')
im2 = cv2.imread('png_from_pdf.png')
# decodedObjects = decode(im2)
# display(im2, decodedObjects)

In [85]:
doc = fitz.open(path)
pm = doc[0].getPixmap(matrix=fitz.Matrix(2,2), alpha=False)
image_data = pm.getImageData()
# image_data = pm.getPNGData()
nparr = np.frombuffer(image_data, np.uint8)
img_np = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
# cv2.imshow("Results", img_np)
# cv2.waitKey(0);
decodedObjects = decode(img_np)
display(img_np, decodedObjects)

Type :  QRCODE
Data :  01,10,021001900111,23763823,85.33,20190322,70446058541415909133,E4CD, 



Type :  QRCODE
Data :  01,10,021001900111,23763823,85.33,20190322,70446058541415909133,E4CD, 



In [152]:
#coding: utf-8
import fitz
# from zbarlight import scan_codes
import pyzbar.pyzbar as pyzbar
from io import BytesIO
from PIL import Image
from datetime import datetime
import os
import re
import subprocess
import numpy as np

class InvoiceExtraction:
    qrcode_keys = [
        ['f0', 'f1', '发票代码', '发票号码', '金额', '开票日期', '校验码', 'f6'],
        ['f0', 'f1', '发票代码', '发票号码', '销售方纳税人识别号', '金额', '开票日期', '校验码'],
    ]

    regex_element = {
        '开票地': re.compile(r'(?P<field>.*增值税电子((普通)|(专用))发票)'),
        '价税合计(小写)': re.compile(r'[\(（]小写[\)）](.*\n)*?[¥￥](?P<field>\d+\.\d+)', re.M),
        '密码区': re.compile(r'(?P<field>(([0-9/+\-\<\>\*]\s?){26,27}[0-9/+\-\<\>\*]\n){4})', re.M),
        '价税合计(大写)': re.compile(r'(?P<field>[壹贰叁肆伍陆柒捌玖拾]\s?[零壹贰叁肆伍陆柒捌玖拾佰仟万亿整元圆角分\s]+[整元圆角分])'),
        '购买方纳税人识别号': re.compile(r'纳税人识别号:\n(.*\n)*?(?P<field>[0-9a-zA-Z]{18})', re.M)
    }

    def extract_qrcode_info(self, file_path):
        try:
            doc = fitz.open(file_path)
            png = doc[0].getPixmap(matrix=fitz.Matrix(2,2), alpha=False).getPNGdata()
#             img = Image.open(BytesIO(png))
#             values = filter(None, scan_codes(['qrcode'], img)[0].decode().split(','))
            nparr = np.frombuffer(png, np.uint8)
            img_np = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
            decodedObjects = pyzbar.decode(img_np)
            values = decodedObjects[0].data.decode(encoding='utf-8').split(',')
            print(values)
            for keys in self.qrcode_keys:
                ret = dict(zip(keys, values))
                try:
                    ret['金额'] = eval(ret['金额'])
                    ret['开票日期'] = datetime.strptime(ret['开票日期'], '%Y%m%d').date()
                except:
                    pass
                else:
                    break
            for key in ['f0', 'f1', 'f6']:
                ret.pop(key, None)
            ret['values'] = values
            return ret
        except Exception as e:
            print(e)
            return {}
    
    def extract_pdf_info(self, file_path):
        try:
#             text = subprocess.Popen(['pdf2txt.py', file_path], stdout=subprocess.PIPE).stdout.read().decode('utf-8').replace('\n\n','\n').strip()
#             text = subprocess.Popen(['python', 'D:\\Anaconda3\\envs\\myenv\\Scripts\\pdf2txt.py', file_path], stdout=subprocess.PIPE).stdout.read().decode('utf-8').replace('\n\n','\n').strip()
#             result = parse_pdf(file_path)
#             text = result['pages'][0].extract_text()
            text = parse_pdf_with_PdfMiner3K(path)
            
            print(text) 
            
            ret = {}
            for key in self.regex_element:
                mt = self.regex_element[key].search(text)
                if mt:
                    ret[key] = mt.groupdict()['field']
            ret['价税合计(小写)'] = eval(ret.get('价税合计(小写)','None'))
            ret['密码区'] = ret.get('密码区','').replace(' ','').replace('\n','')
            ret['价税合计(大写)'] = ret.get('价税合计(大写)','').replace(' ','')
            return ret
        except Exception as e:
            print(e)
            return {}

    def extract(self, file_path):
        ret = dict(self.extract_pdf_info(file_path), **self.extract_qrcode_info(file_path))
        ret['文件名'] = os.path.split(file_path)[1]
        return ret

In [153]:
pdf_file_name = '日用品20190322_98p99'
#     pdf_file_name = '日用品20190417_197p98'
#     pdf_file_name = '食品20190408_91p8'
path = 'data\\invoice\\{pdf_file_name}.pdf'.format(pdf_file_name=pdf_file_name)

In [154]:
ie = InvoiceExtraction()

In [155]:
# ret = ie.extract(path)
result = ie.extract_pdf_info(path)

 辽宁增值税电子普通发票
监票发一统
制
国
全
国
家税务
总
局
章
宁辽
省税务
局
发票代码:
021001900111
发票号码:
23763823
开票日期:
2019  03  22
年
月
日
机器编号:
661807100422
北京慧远云程科技有限公司
91110105306602928Y
购
买
方
名        称:
纳税人识别号:
地 址、电 话:
开户行及账号:
校 验 码:
70446  05854  14159  09133  
59+8>9232--*+99+333*-<9/1/-
60/4</2*/137<6/0110*0/2<2</
/0+9</9*<+*6*1+1+19>+130>-9
*09>12>5+/1+/0-><>/0998+95+
密
码
区
货物或应税劳务、服务名称      规格型号   单位
 数 量
   单 价
金  额
  税率          税  额
*纺织产品*天石（HighRock）户外
露营情侣左右互拼成人棉睡袋 -10度 
右开 天蓝色
*纺织产品*天石（HighRock）户外
露营情侣左右互拼成人棉睡袋 -10度 
右开 天蓝色
冷杉
1
171.55
171.55
16%
-86.22
16%
合       计
价税合计(大写)
玖拾捌圆玖角玖分
¥85.33
(小写)
¥98.99
销
售
方
名        称:
纳税人识别号:
地 址、电 话:
开户行及账号:
沈阳京东世纪贸易有限公司
912101125893550575
辽宁省沈阳市浑南区麦子屯590-4号102室 024-67859504
中国建设银行沈阳市中山支行 21001360008059003247
备
注
订单号:90235270381
收款人:
京东商城
复核:
开票人:
京东商城
销售方:(章)
27.45
-13.79
¥13.66
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 



In [156]:
pdf_path = 'E:\\workspace201809\\qInvoiceExtractor\\data\\invoice\\日用品20190407_32p8.pdf'
ret = ie.extract(path)

 辽宁增值税电子普通发票
监票发一统
制
国
全
国
家税务
总
局
章
宁辽
省税务
局
发票代码:
021001900111
发票号码:
23763823
开票日期:
2019  03  22
年
月
日
机器编号:
661807100422
北京慧远云程科技有限公司
91110105306602928Y
购
买
方
名        称:
纳税人识别号:
地 址、电 话:
开户行及账号:
校 验 码:
70446  05854  14159  09133  
59+8>9232--*+99+333*-<9/1/-
60/4</2*/137<6/0110*0/2<2</
/0+9</9*<+*6*1+1+19>+130>-9
*09>12>5+/1+/0-><>/0998+95+
密
码
区
货物或应税劳务、服务名称      规格型号   单位
 数 量
   单 价
金  额
  税率          税  额
*纺织产品*天石（HighRock）户外
露营情侣左右互拼成人棉睡袋 -10度 
右开 天蓝色
*纺织产品*天石（HighRock）户外
露营情侣左右互拼成人棉睡袋 -10度 
右开 天蓝色
冷杉
1
171.55
171.55
16%
-86.22
16%
合       计
价税合计(大写)
玖拾捌圆玖角玖分
¥85.33
(小写)
¥98.99
销
售
方
名        称:
纳税人识别号:
地 址、电 话:
开户行及账号:
沈阳京东世纪贸易有限公司
912101125893550575
辽宁省沈阳市浑南区麦子屯590-4号102室 024-67859504
中国建设银行沈阳市中山支行 21001360008059003247
备
注
订单号:90235270381
收款人:
京东商城
复核:
开票人:
京东商城
销售方:(章)
27.45
-13.79
¥13.66
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

['01', '10', '02100

In [157]:
ret

{'开票地': ' 辽宁增值税电子普通发票',
 '价税合计(小写)': 98.99,
 '密码区': '59+8>9232--*+99+333*-<9/1/-60/4</2*/137<6/0110*0/2<2<//0+9</9*<+*6*1+1+19>+130>-9*09>12>5+/1+/0-><>/0998+95+',
 '价税合计(大写)': '玖拾捌圆玖角玖分',
 '购买方纳税人识别号': '912101125893550575',
 '发票代码': '021001900111',
 '发票号码': '23763823',
 '金额': 85.33,
 '开票日期': datetime.date(2019, 3, 22),
 '校验码': '70446058541415909133',
 'values': ['01',
  '10',
  '021001900111',
  '23763823',
  '85.33',
  '20190322',
  '70446058541415909133',
  'E4CD',
  ''],
 '文件名': '日用品20190322_98p99.pdf'}

In [158]:
qrcode_info = ie.extract_qrcode_info(path)

['01', '10', '021001900111', '23763823', '85.33', '20190322', '70446058541415909133', 'E4CD', '']


In [159]:
qrcode_info

{'发票代码': '021001900111',
 '发票号码': '23763823',
 '金额': 85.33,
 '开票日期': datetime.date(2019, 3, 22),
 '校验码': '70446058541415909133',
 'values': ['01',
  '10',
  '021001900111',
  '23763823',
  '85.33',
  '20190322',
  '70446058541415909133',
  'E4CD',
  '']}

In [239]:
result2

' 辽宁增值税电子普通发票\n监票发一统\n制\n国\n全\n国\n家税务\n总\n局\n章\n宁辽\n省税务\n局\n发票代码:\n021001900111\n发票号码:\n23763823\n开票日期:\n2019  03  22\n年\n月\n日\n机器编号:\n661807100422\n北京慧远云程科技有限公司\n91110105306602928Y\n购\n买\n方\n名        称:\n纳税人识别号:\n地 址、电 话:\n开户行及账号:\n校 验 码:\n70446  05854  14159  09133  \n59+8>9232--*+99+333*-<9/1/-\n60/4</2*/137<6/0110*0/2<2</\n/0+9</9*<+*6*1+1+19>+130>-9\n*09>12>5+/1+/0-><>/0998+95+\n密\n码\n区\n货物或应税劳务、服务名称      规格型号   单位\n 数 量\n   单 价\n金  额\n  税率          税  额\n*纺织产品*天石（HighRock）户外\n露营情侣左右互拼成人棉睡袋 -10度 \n右开 天蓝色\n*纺织产品*天石（HighRock）户外\n露营情侣左右互拼成人棉睡袋 -10度 \n右开 天蓝色\n冷杉\n1\n171.55\n171.55\n16%\n-86.22\n16%\n合       计\n价税合计(大写)\n玖拾捌圆玖角玖分\n¥85.33\n(小写)\n¥98.99\n销\n售\n方\n名        称:\n纳税人识别号:\n地 址、电 话:\n开户行及账号:\n沈阳京东世纪贸易有限公司\n912101125893550575\n辽宁省沈阳市浑南区麦子屯590-4号102室 024-67859504\n中国建设银行沈阳市中山支行 21001360008059003247\n备\n注\n订单号:90235270381\n收款人:\n京东商城\n复核:\n开票人:\n京东商城\n销售方:(章)\n27.45\n-13.79\n¥13.66\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \

In [243]:
text_pdfplumber = result['pages'][0].extract_text()
text_pdfplumber

'     发票代码:021001900111\n      辽宁增值税电子普通发票\n     国 统 一 发票监 发票号码:23763823\n全 制\n                 国家税务总局 章 开票日期:2019 年 03 月 22日\n     机器编号: 661807100422 辽宁省税务局 校 验 码:70446  05854  14159  09133  \n59+8>9232--*+99+333*-<9/1/-\n     名        称:北京慧远云程科技有限公司\n     购 91110105306602928Y 密 60/4</2*/137<6/0110*0/2<2</\n     纳税人识别号:\n     买 码 /0+9</9*<+*6*1+1+19>+130>-9\n     地 址、电 话:\n方 *09>12>5+/1+/0-><>/0998+95+\n     区\n     开户行及账号:\n                         货物或应税劳务、服务名称      规格型号   单位  数 量    单 价 金  额   税率        税    额\n*纺织产品*天石（HighRock）户外 冷杉 1 171.55 171.55 16% 27.45\n露营情侣左右互拼成人棉睡袋 -10度 \n右开 天蓝色\n*纺织产品*天石（HighRock）户外 -86.22 16% -13.79\n露营情侣左右互拼成人棉睡袋 -10度 \n右开 天蓝色\n¥85.33 ¥13.66\n合       计\n¥98.99\n价税合计(大写) 玖拾捌圆玖角玖分 (小写)\n名        称:沈阳京东世纪贸易有限公司 订单号:90235270381\n销 912101125893550575 备\n纳税人识别号:\n售\n地 址、电 话:辽宁省沈阳市浑南区麦子屯590-4号102室 024-67859504\n方 注\n开户行及账号:中国建设银行沈阳市中山支行 21001360008059003247\n收款人:京东商城 复核: 开票人:京东商城 销售方:(章)'

In [228]:
test_regex_element = {
        '开票地': re.compile(r'(?P<field>.*增值税电子((普通)|(专用))发票)'),
        '价税合计(小写)': re.compile(r'[\(（]小写[\)）](.*\n)*?[¥￥](?P<field>\d+\.\d+)', re.M),
        '密码区': re.compile(r'(?P<field>(([0-9/+\-\<\>\*]\s?){26,27}[0-9/+\-\<\>\*]\n){4})', re.M),
        '价税合计(大写)': re.compile(r'(?P<field>[壹贰叁肆伍陆柒捌玖拾]\s?[零壹贰叁肆伍陆柒捌玖拾佰仟万亿整元圆角分\s]+[整元圆角分])'),
        '购买方纳税人识别号': re.compile(r'纳税人识别号:\n(.*\n)*?(?P<field>[0-9a-zA-Z]{18})', re.M)
    }

In [249]:
test_mt = test_regex_element['购买方纳税人识别号'].search(text_pdfplumber)
test_mt

In [235]:
test_mt

<re.Match object; span=(566, 574), match='玖拾捌圆玖角玖分'>

In [251]:
test_mt2 = buyer_id_regex.search(text_pdfplumber)

In [252]:
test_mt2

In [253]:
test_mt2.groupdict()['buyerID']

AttributeError: 'NoneType' object has no attribute 'groupdict'